In [1]:
import pandas as pd
import numpy as np

properties_2016 = pd.read_csv("../data/properties_2016.csv")
train_2016 = pd.read_csv("../data/train_2016_v2.csv")
# should only be training on properties that actually sold
df = train_2016.merge(properties_2016, on='parcelid', how='left')

print("Training data shape:", train_2016.shape)
print("Properties data shape:", properties_2016.shape)
print("Merged data shape:", df.shape)
df.head()

C:\Users\gkamt\AppData\Local\Temp\ipykernel_27148\3418956264.py:4: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  properties_2016 = pd.read_csv("../data/properties_2016.csv")


Training data shape: (90275, 3)
Properties data shape: (2985217, 58)
Merged data shape: (90275, 60)


,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [2]:
# find out how empty the most empty columns are
missing_pct = df.isnull().sum() / len(df)
print(missing_pct.sort_values(ascending=False))

buildingclasstypeid             0.999823
finishedsquarefeet13            0.999634
storytypeid                     0.999524
basementsqft                    0.999524
yardbuildingsqft26              0.998948
fireplaceflag                   0.997541
architecturalstyletypeid        0.997109
typeconstructiontypeid          0.996688
finishedsquarefeet6             0.995336
decktypeid                      0.992711
poolsizesum                     0.989266
pooltypeid10                    0.987139
pooltypeid2                     0.986663
taxdelinquencyyear              0.980249
taxdelinquencyflag              0.980249
hashottuborspa                  0.973802
yardbuildingsqft17              0.970690
finishedsquarefeet15            0.960521
finishedfloor1squarefeet        0.924054
finishedsquarefeet50            0.924054
fireplacecnt                    0.893581
threequarterbathnbr             0.866973
pooltypeid7                     0.815043
poolcnt                         0.801706
numberofstories 

In [3]:
# drop columns that are more than 85% empty
missing_pct = df.isnull().sum() / len(df)
columns_to_drop = missing_pct[missing_pct > 0.85].index
df = df.drop(columns=columns_to_drop)

print(f"Dropped {len(columns_to_drop)} columns")
print(f"Remaining columns: {df.shape[1]}")
df.info()

Dropped 22 columns
Remaining columns: 38
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      90275 non-null  int64  
 1   logerror                      90275 non-null  float64
 2   transactiondate               90275 non-null  object 
 3   airconditioningtypeid         28781 non-null  float64
 4   bathroomcnt                   90275 non-null  float64
 5   bedroomcnt                    90275 non-null  float64
 6   buildingqualitytypeid         57364 non-null  float64
 7   calculatedbathnbr             89093 non-null  float64
 8   calculatedfinishedsquarefeet  89614 non-null  float64
 9   finishedsquarefeet12          85596 non-null  float64
 10  fips                          90275 non-null  float64
 11  fullbathcnt                   89093 non-null  float64
 12  garagecarcnt       

In [4]:
# convert date from object to datetime
df['transactiondate'] = pd.to_datetime(df['transactiondate'])
df['transaction_month'] = df['transactiondate'].dt.month
df['transaction_day'] = df['transactiondate'].dt.day
df['transaction_quarter'] = df['transactiondate'].dt.quarter
df['transaction_dayofweek'] = df['transactiondate'].dt.dayofweek
# we've extracted the important aspects of the data, can drop the actual date
df = df.drop('transactiondate', axis=1)
df = df.drop('assessmentyear', axis=1)

# convert propertycountylanduse codes to numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['landusecode_encoded'] = le.fit_transform(df['propertycountylandusecode'])
# drop object version of the row
df = df.drop('propertycountylandusecode', axis=1)

# handle asterisk in property zoning description column
df['zoning_special_case'] = df['propertyzoningdesc'].str.contains(r'\*', na=False, regex=True)
df['propertyzoningdesc_clean'] = df['propertyzoningdesc'].str.replace('*', '', regex=False)
le_zoning = LabelEncoder()
df['zoning_encoded'] = le_zoning.fit_transform(df['propertyzoningdesc_clean'].fillna('Unknown'))
# drop original columns
df = df.drop('propertyzoningdesc', axis=1)
df = df.drop('propertyzoningdesc_clean', axis=1)

# removes rows missing critical fields
df = df.dropna(subset=['bathroomcnt', 'bedroomcnt', 'taxamount', 'taxvaluedollarcnt'])
df = df[df['bedroomcnt'] > 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88847 entries, 0 to 90274
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      88847 non-null  int64  
 1   logerror                      88847 non-null  float64
 2   airconditioningtypeid         28595 non-null  float64
 3   bathroomcnt                   88847 non-null  float64
 4   bedroomcnt                    88847 non-null  float64
 5   buildingqualitytypeid         57079 non-null  float64
 6   calculatedbathnbr             88808 non-null  float64
 7   calculatedfinishedsquarefeet  88838 non-null  float64
 8   finishedsquarefeet12          85202 non-null  float64
 9   fips                          88847 non-null  float64
 10  fullbathcnt                   88808 non-null  float64
 11  garagecarcnt                  29727 non-null  float64
 12  garagetotalsqft               29727 non-null  float64
 13  heatin

In [5]:
# begin imputing missing data

# assume no pool if one isn't indicated
df['poolcnt'] = df['poolcnt'].fillna(0)
df['pooltypeid7'] = df['pooltypeid7'].fillna(0)

# impute bathroom counts based on median sqft for same bedroom count
df['bathroomcnt'] = df.groupby('bedroomcnt')['bathroomcnt'].transform(lambda x: x.fillna(x.median()))
df['bathroomcnt'] = df['bathroomcnt'].fillna(df['bathroomcnt'].median())
df['calculatedbathnbr'] = df['calculatedbathnbr'].fillna(df['bathroomcnt'])

# impute square footage based on median sqft for same bedroom count
df['calculatedfinishedsquarefeet'] = df.groupby('bedroomcnt')['calculatedfinishedsquarefeet'].transform(lambda x: x.fillna(x.median()))
df['calculatedfinishedsquarefeet'] = df['calculatedfinishedsquarefeet'].fillna(df['calculatedfinishedsquarefeet'].median())
df['finishedsquarefeet12'] = df.groupby('bedroomcnt')['finishedsquarefeet12'].transform(lambda x: x.fillna(x.median()))
df['finishedsquarefeet12'] = df['finishedsquarefeet12'].fillna(df['finishedsquarefeet12'].median())
print(df.isnull().sum())

parcelid                            0
logerror                            0
airconditioningtypeid           60252
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           31768
calculatedbathnbr                   0
calculatedfinishedsquarefeet        0
finishedsquarefeet12                0
fips                                0
fullbathcnt                        39
garagecarcnt                    59120
garagetotalsqft                 59120
heatingorsystemtypeid           33031
latitude                            0
longitude                           0
lotsizesquarefeet                9796
poolcnt                             0
pooltypeid7                         0
propertylandusetypeid               0
rawcensustractandblock              0
regionidcity                     1692
regionidcounty                      0
regionidneighborhood            53385
regionidzip                        15
roomcnt                             0
unitcnt     

In [6]:
# estimate fullbathcnt from bathroomcnt
df['fullbathcnt'] = df['fullbathcnt'].fillna(df['bathroomcnt'].apply(lambda x: max(1, int(x)) if x > 0 else 0))

# assume missing HVAC features = none
df['airconditioningtypeid'] = df['airconditioningtypeid'].fillna(0)
df['heatingorsystemtypeid'] = df['heatingorsystemtypeid'].fillna(0)

# if garagecarcnt is missing but garagetotalsqft exists, base count based on sqft
# typical single garage is 400 sqft; double car garage 800 sqft
df.loc[df['garagecarcnt'].isna() & df['garagetotalsqft'].notna(), 'garagecarcnt'] = (
    df.loc[df['garagecarcnt'].isna() & df['garagetotalsqft'].notna(), 'garagetotalsqft'] / 400).round()
# if garagetotalsqft is missing but garagecarcnt exists, use median sqft for that count
df['garagetotalsqft'] = df.groupby('garagecarcnt')['garagetotalsqft'].transform(lambda x: x.fillna(x.median()))
# if both still missing, impute based on bedroom count
# based on logic that more bedrooms typically = more likely to have garage
df['garagecarcnt'] = df.groupby('bedroomcnt')['garagecarcnt'].transform(lambda x: x.fillna(x.median()))
df['garagetotalsqft'] = df.groupby('bedroomcnt')['garagetotalsqft'].transform(lambda x: x.fillna(x.median()))
# final fallback = 0
df['garagecarcnt'] = df['garagecarcnt'].fillna(0)
df['garagetotalsqft'] = df['garagetotalsqft'].fillna(0)
print(df.isnull().sum())

parcelid                            0
logerror                            0
airconditioningtypeid               0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           31768
calculatedbathnbr                   0
calculatedfinishedsquarefeet        0
finishedsquarefeet12                0
fips                                0
fullbathcnt                         0
garagecarcnt                        0
garagetotalsqft                     0
heatingorsystemtypeid               0
latitude                            0
longitude                           0
lotsizesquarefeet                9796
poolcnt                             0
pooltypeid7                         0
propertylandusetypeid               0
rawcensustractandblock              0
regionidcity                     1692
regionidcounty                      0
regionidneighborhood            53385
regionidzip                        15
roomcnt                             0
unitcnt     

In [7]:
df = df.drop('buildingqualitytypeid', axis=1)

# geographic hierarchy is county --> city --> zip --> neighborhood
# impute missing values based on other geographic IDs
df['regionidcity'] = df.groupby('regionidzip')['regionidcity'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
df['regionidzip'] = df.groupby('regionidcity')['regionidzip'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
df['regionidneighborhood'] = df.groupby(['regionidcity', 'regionidzip'])['regionidneighborhood'].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
df['regionidcity'] = df.groupby('regionidcounty')['regionidcity'].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)

# cities should cluster based on latitude and longitude
df['regionidcity'] = df['regionidcity'].fillna(df['regionidcity'].mode()[0])
df['regionidzip'] = df['regionidzip'].fillna(df['regionidzip'].mode()[0])

df['regionidneighborhood'] = df['regionidneighborhood'].fillna(0)
print(df.isnull().sum())

parcelid                            0
logerror                            0
airconditioningtypeid               0
bathroomcnt                         0
bedroomcnt                          0
calculatedbathnbr                   0
calculatedfinishedsquarefeet        0
finishedsquarefeet12                0
fips                                0
fullbathcnt                         0
garagecarcnt                        0
garagetotalsqft                     0
heatingorsystemtypeid               0
latitude                            0
longitude                           0
lotsizesquarefeet                9796
poolcnt                             0
pooltypeid7                         0
propertylandusetypeid               0
rawcensustractandblock              0
regionidcity                        0
regionidcounty                      0
regionidneighborhood                0
regionidzip                         0
roomcnt                             0
unitcnt                         31151
yearbuilt   

In [8]:
# impute based on bedroom count and building sqft relationship
df['lotsizesquarefeet'] = df.groupby('bedroomcnt')['lotsizesquarefeet'].transform(lambda x: x.fillna(x.median()))
# fallback to overall median
df['lotsizesquarefeet'] = df['lotsizesquarefeet'].fillna(df['lotsizesquarefeet'].median())

# use median by neighborhood/city since building age clusters geographically
df['yearbuilt'] = df.groupby('regionidcity')['yearbuilt'].transform(lambda x: x.fillna(x.median()))
# fallback to overall median
df['yearbuilt'] = df['yearbuilt'].fillna(df['yearbuilt'].median())

# impute based on property land use type
df['unitcnt'] = df.groupby('propertylandusetypeid')['unitcnt'].transform(lambda x: x.fillna(x.median()))
# otherwise impute based on zoning
df['unitcnt'] = df.groupby('zoning_encoded')['unitcnt'].transform(lambda x: x.fillna(x.median()))
# otherwise impute based on size only
df['size_category'] = pd.cut(df['calculatedfinishedsquarefeet'], bins=[0, 1000, 2000, 3000, 10000], labels=['small', 'medium', 'large', 'xlarge'])
df['unitcnt'] = df.groupby('size_category')['unitcnt'].transform(lambda x: x.fillna(x.median()))
# final fallback to 1 (single-family)
df['unitcnt'] = df['unitcnt'].fillna(1)

df = df.drop('size_category', axis=1)
df = df.drop('numberofstories', axis=1) # too sparse
print(df.isnull().sum())

parcelid                          0
logerror                          0
airconditioningtypeid             0
bathroomcnt                       0
bedroomcnt                        0
calculatedbathnbr                 0
calculatedfinishedsquarefeet      0
finishedsquarefeet12              0
fips                              0
fullbathcnt                       0
garagecarcnt                      0
garagetotalsqft                   0
heatingorsystemtypeid             0
latitude                          0
longitude                         0
lotsizesquarefeet                 0
poolcnt                           0
pooltypeid7                       0
propertylandusetypeid             0
rawcensustractandblock            0
regionidcity                      0
regionidcounty                    0
regionidneighborhood              0
regionidzip                       0
roomcnt                           0
unitcnt                           0
yearbuilt                         0
structuretaxvaluedollarcnt  

C:\Users\gkamt\AppData\Local\Temp\ipykernel_27148\1492886583.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['unitcnt'] = df.groupby('size_category')['unitcnt'].transform(lambda x: x.fillna(x.median()))


In [9]:
# estimate as about 70% of total tax value
df['structuretaxvaluedollarcnt'] = df['structuretaxvaluedollarcnt'].fillna(df['taxvaluedollarcnt'] * 0.7)

# use rawcensustractandblock to impute
df['censustractandblock'] = df['censustractandblock'].fillna(df['rawcensustractandblock'])

print(df.isnull().sum())

parcelid                        0
logerror                        0
airconditioningtypeid           0
bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
heatingorsystemtypeid           0
latitude                        0
longitude                       0
lotsizesquarefeet               0
poolcnt                         0
pooltypeid7                     0
propertylandusetypeid           0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidneighborhood            0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedo

In [10]:
df.to_csv('imputed.csv', index=False) 